In [1]:
print("SFTTrainer example mac")

SFTTrainer example mac


In [25]:
# imports
from dataclasses import dataclass, field
from typing import Optional

from peft import LoraConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, HfArgumentParser
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
import wandb

In [6]:
full_dataset = load_dataset("lucasmccabe-lmi/openai_humaneval_alpaca_style", split="train")

Generating train split: 100%|██████████| 164/164 [00:00<00:00, 20693.92 examples/s]


In [8]:
full_dataset[0]

{'instruction': '\ndef multiply(a, b):\n    """Complete the function that takes two integers and returns \n    the product of their unit digits.\n    Assume the input is always valid.\n    Examples:\n    multiply(148, 412) should return 16.\n    multiply(19, 28) should return 72.\n    multiply(2020, 1851) should return 0.\n    multiply(14,-15) should return 20.\n    """\n',
 'input': "The prompt's code is written in Python. Write corresponding response code in Python, as well.",
 'output': '    return abs(a % 10) * abs(b % 10)\n'}

In [9]:
full_dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 164
})

In [10]:
dataset_dict = full_dataset.train_test_split(test_size=0.1)

In [11]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 147
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 17
    })
})

In [12]:
train_dataset = dataset_dict["train"]
eval_dataset = dataset_dict["test"]

In [13]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

/Users/bihan/miniconda3/envs/machine-learning-book/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["k_proj", "v_proj"],
)

In [17]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

In [18]:
trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=SFTConfig(output_dir="/tmp", max_steps=5, per_device_train_batch_size=1,
                   per_device_eval_batch_size=1),
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
)

Map: 100%|██████████| 17/17 [00:00<00:00, 3246.81 examples/s]
max_steps is given, it will override any value given in num_train_epochs


In [26]:
trainer.train()

Step,Training Loss


AttributeError: module 'wandb' has no attribute 'run'

In [21]:
dataset_imdb = load_dataset("imdb", split="train")

Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 820761.37 examples/s]


In [22]:
dataset_imdb

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [28]:
dataset_imdb[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [31]:

sft_config = SFTConfig(
    dataset_text_field="text",
    # max_seq_length=512,
    output_dir="/tmp",
)
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset_imdb,
    args=sft_config,
)
trainer.train()

Map: 100%|██████████| 25000/25000 [00:03<00:00, 6789.92 examples/s]


AttributeError: module 'wandb' has no attribute 'termwarn'